# Lab 3 — recommender systems

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pickle
from operator import itemgetter
%matplotlib inline
plt.style.use("ggplot")

What does the data look like?

In [ ]:
!hdfs dfs -cat /ix/ml-20m/ratings.txt | tail -n 2

In [ ]:
data = sc.textFile("/ix/ml-20m/ratings.txt").map(json.loads)
user = data.map(itemgetter('userId'))
user = user.map(lambda x: (x,1))
user = user.reduceByKey(lambda a,b: a+b)
item = data.map(itemgetter('movieId'))
item = item.map(lambda x: (x,1))
item = item.reduceByKey(lambda a,b:a+b)
item = item.map(lambda x: x[1])
user = user.map(lambda x:x[1])

In [ ]:
x = (user.collect())
y = (item.collect())
plt.plot(sorted(x))
plt.show()
plt.plot(sorted(y))
plt.show()


We can see that the ratings are not balanced uniformly accros users and movies

In [ ]:
%run rate-movies.py

In [ ]:
!hdfs dfs -put my-ratings.txt

In [ ]:
#Create trainig and validation st
myRates = sc.textFile('my-ratings.txt').map(json.loads)
newData = data.union(myRates)
count = newData.count()
trainingSet = data.filter(lambda x: x['timestamp']%10 <= 7)
validationSet = data.filter(lambda x: x['timestamp']%10 > 7)
mean = trainingSet.map(lambda x: x['rating']).mean()



In [ ]:
#Compute user bias
userBias = trainingSet.map(lambda x:(x['userId'],x['rating']-mean)).groupByKey().map(lambda x:(x[0],sum(x[1])/len(x[1]))).collect()

userBiasDic = {}
for item in userBias:
    userBiasDic[item[0]] = item[1]
users = newData.map(lambda x: x['userId']).distinct().collect()
for user in users:
    if user not in userBiasDic.keys():
        userBiasDic[user] = 0
#Compute item bias
itemBias = trainingSet.map(lambda x: (x['movieId'],x['rating']-mean-userBiasDic[x['userId']])).groupByKey().map(lambda x:(x[0],sum(x[1])/len(x[1]))).sortByKey().collect()
itemBiasDic = {}
for item in itemBias:
    itemBiasDic[item[0]] = item[1] 
movies = newData.map(lambda x: x['movieId']).distinct().collect()
for movie in movies:
    if movie not in itemBiasDic.keys():
        itemBiasDic[movie] = 0


In [ ]:
#Compute predicted ratings
rating = validationSet.map(lambda x: (x['movieId'],x['userId'],mean+userBiasDic[x['userId']]+itemBiasDic[x['movieId']])).collect()
ratingDic = {}
for item in rating:
    ratingDic[(item[0],item[1])] = item[2]  



In [ ]:
def error(ratings):
    print(ratings)
    temp = validationSet.map(lambda x:(x['userId'],pow(ratings[(x['movieId'],x['userId'])]-x['rating'],2))).groupByKey().map(lambda x:sum(x[1])/len(x[1])).sum()
    nbrUser = validationSet.map(lambda x:x['userId']).distinct().count()
    err = temp / nbrUser
    return err

In [ ]:

print(error(ratingDic))

0.8500001435

With a low rank, the result is less precise but the computation is fast. It is the opposite with high rank

In [ ]:
#Try to find the best lambda
from pyspark.mllib.recommendation import ALS
from pyspark.mllib.recommendation import MatrixFactorizationModel
lambdas = np.arange(0.01,0.1,0.01)
minErr = 500
bestLambda = 0
for lamb in lambdas:
    matrix = ALS.train(newData.map(lambda x:(x['userId'],x['movieId'],x['rating'])),12,lambda_=lamb)
    result = matrix.predictAll(validationSet.map(lambda x:(x['userId'],x['movieId']))).collect()
    resultDic = {}
    for item in result:
        resultDic[(item[1],item[0])] = item[2]
    err = error(resultDic)
    print(err," with lambda =",lamb)
    if err<minErr:
        bestLambda = lamb
        minErr = err
    
    
print(minErr,bestLambda)

In [ ]:
top10For123 = matrix.recommendProducts(123,10)
temp =[]
for item in top10For123:
    temp.append(item[1])
file = sc.textFile("/ix/ml-20m/movies.txt").map(json.loads)
resultMovies =[]
file = file.filter(lambda x: x['movieId'] in temp).map(lambda x: (x['title'],x['genres'])).collect()
print(file)

We can see that the user 123 likes the drama movies